In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from sklearn.metrics import classification_report, confusion_matrix

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data_dir = "/kaggle/input/chest-xray-pneumonia/chest_xray"  # Kaggle path
# or path to dataset locally

batch_size = 32
img_size = 224   # for ResNet
num_workers = 4  # reduce if on CPU only

train_transforms = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.1, contrast=0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

val_test_transforms = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])


In [3]:
train_dir = os.path.join(data_dir, "train")
val_dir   = os.path.join(data_dir, "val")
test_dir  = os.path.join(data_dir, "test")

train_dataset = datasets.ImageFolder(train_dir, transform=train_transforms)
val_dataset   = datasets.ImageFolder(val_dir,   transform=val_test_transforms)
test_dataset  = datasets.ImageFolder(test_dir,  transform=val_test_transforms)

class_names = train_dataset.classes
print("Classes:", class_names)  # ['NORMAL', 'PNEUMONIA']

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                          num_workers=num_workers)
val_loader   = DataLoader(val_dataset,   batch_size=batch_size, shuffle=False,
                          num_workers=num_workers)
test_loader  = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False,
                          num_workers=num_workers)


Classes: ['NORMAL', 'PNEUMONIA']


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)

# freeze backbone initially (optional)
for param in model.parameters():
    param.requires_grad = False

# replace final layer
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)  # 2 classes

model = model.to(device)

Device: cuda


In [ ]:
criterion = nn.CrossEntropyLoss()

# only train final layer at first
optimizer = optim.Adam(model.fc.parameters(), lr=1e-3)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min',
                                                 factor=0.1, patience=3)

In [12]:
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler,
                num_epochs=10):
    best_val_acc = 0.0
    best_state_dict = None

    for epoch in range(num_epochs):
        # training
        model.train()
        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            _, preds = torch.max(outputs, 1)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = running_corrects.double() / len(train_loader.dataset)

        # validation
        model.eval()
        val_loss = 0.0
        val_corrects = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                _, preds = torch.max(outputs, 1)
                val_loss += loss.item() * inputs.size(0)
                val_corrects += torch.sum(preds == labels.data)

        val_loss /= len(val_loader.dataset)
        val_acc = val_corrects.double() / len(val_loader.dataset)
        scheduler.step(val_loss)

        print(f"Epoch {epoch+1}/{num_epochs} "
              f"Train loss: {epoch_loss:.4f}, acc: {epoch_acc:.4f} | "
              f"Val loss: {val_loss:.4f}, acc: {val_acc:.4f}")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_state_dict = model.state_dict()

    if best_state_dict is not None:
        model.load_state_dict(best_state_dict)

    print("Best val acc:", best_val_acc.item())
    return model

model = train_model(model, train_loader, val_loader, criterion, optimizer,
                    scheduler, num_epochs=10)


Epoch 1/10 Train loss: 0.3253, acc: 0.8604 | Val loss: 0.3447, acc: 0.8125
Epoch 2/10 Train loss: 0.1962, acc: 0.9241 | Val loss: 0.5006, acc: 0.7500
Epoch 3/10 Train loss: 0.1733, acc: 0.9316 | Val loss: 0.2235, acc: 0.9375
Epoch 4/10 Train loss: 0.1604, acc: 0.9337 | Val loss: 0.3588, acc: 0.7500
Epoch 5/10 Train loss: 0.1544, acc: 0.9367 | Val loss: 0.3226, acc: 0.7500
Epoch 6/10 Train loss: 0.1431, acc: 0.9475 | Val loss: 0.4203, acc: 0.7500
Epoch 7/10 Train loss: 0.1428, acc: 0.9431 | Val loss: 0.4971, acc: 0.6875
Epoch 8/10 Train loss: 0.1341, acc: 0.9477 | Val loss: 0.4314, acc: 0.7500
Epoch 9/10 Train loss: 0.1255, acc: 0.9509 | Val loss: 0.4165, acc: 0.7500
Epoch 10/10 Train loss: 0.1274, acc: 0.9498 | Val loss: 0.3282, acc: 0.7500
Best val acc: 0.9375


In [13]:
model.eval()
all_labels = []
all_preds = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())

print("Classification report:")
print(classification_report(all_labels, all_preds,
                            target_names=class_names))

print("Confusion matrix:")
print(confusion_matrix(all_labels, all_preds))


Classification report:
              precision    recall  f1-score   support

      NORMAL       0.94      0.71      0.81       234
   PNEUMONIA       0.85      0.97      0.91       390

    accuracy                           0.88       624
   macro avg       0.89      0.84      0.86       624
weighted avg       0.88      0.88      0.87       624

Confusion matrix:
[[167  67]
 [ 11 379]]
